In [1]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv
!ls

--2021-06-01 05:15:24--  https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv
archive.ics.uci.edu (archive.ics.uci.edu) をDNSに問いあわせています... 128.195.10.252
archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 84199 (82K) [application/x-httpd-php]
`winequality-red.csv' に保存中

winequality-red.csv 100%[===================>]  82.23K   311KB/s 時間 0.3s       

2021-06-01 05:15:25 (311 KB/s) - `winequality-red.csv' へ保存完了 [84199/84199]

Untitled.ipynb      winequality-red.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
#MinMaxScalerによる正則化
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
#クロスバリデーションとは、データを分割して学習、判定を繰り返す検証方法
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
from sklearn.linear_model import Ridge
import io

# データの読み込み
wine = pd.read_csv('winequality-red.csv', header=0,  sep=";")

# 利用する特徴量リスト
featureName=['fixed acidity', 'volatile acidity', 'citric acid', 
             'residual sugar', 'chlorides', 'free sulfur dioxide', 
             'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']

# 予測したい目標値
objectiveName=['quality']

# 特徴量のみのデータを作成
wine_except_quality = wine.drop("quality", axis=1)
X = wine_except_quality.values

# スケーリング
sc = StandardScaler()
X = sc.fit_transform(X)

# 目標値のデータを作成
y = wine['quality'].values

# 正則化パラメータ
alphas = [0]

for a in alphas:
    
    # リッジ回帰のモデルを学習
    #fit_intercept: 切片
    #nomalize: 回帰式計算の前に最適化
    #copy_X: 係数
    #alpha: 正則化パラメータ
    clf = Ridge(fit_intercept=True, normalize=True,  copy_X=True, alpha=a)
    clf.fit(X, y)

    # 訓練誤差
    y_pred=clf.predict(X)
    #個々のターゲットのスコアまたは損失を平均する方法を指定: uniform_average
    #uniform_average: 出力に対して均一に重み付けされた平均を指定
    mse=mean_squared_error(y, y_pred, multioutput='uniform_average')

    # 交差検証によるテスト誤差の評価
    #cv: データの分割数
    scores = cross_val_score(clf, X, y, scoring="neg_mean_squared_error", cv=10)
    
    # モデルのパラメータ
    print(pd.DataFrame({"Name":wine_except_quality.columns,
                   "Coefficients":clf.coef_}).sort_values(by='Coefficients') ,"\n")
 
    #results
    print( "Mean squared err. ", mse, "\n")



                    Name  Coefficients
1       volatile acidity     -0.193967
6   total sulfur dioxide     -0.107356
4              chlorides     -0.088183
8                     pH     -0.063842
2            citric acid     -0.035553
7                density     -0.033737
3         residual sugar      0.023019
0          fixed acidity      0.043497
5    free sulfur dioxide      0.045606
9              sulphates      0.155277
10               alcohol      0.294243 

Mean squared err.  0.41676716722140794 

